In [2]:
import pandas as pd
cost = pd.DataFrame([[20,18,21,8],[8,23,24,8],[25,8,8,19]],\
                    index = [1,2,3],columns=['A','B','C','D'])
cost

,A,B,C,D
1,20,18,21,8
2,8,23,24,8
3,25,8,8,19


In [7]:
for f in cost.index:
    for r in cost.columns:
        print(cost.loc[f,r],end=' ')
    print()

20 18 21 8 
8 23 24 8 
25 8 8 19 


In [11]:
demand = pd.Series([30,50,10,20],index=['A','B','C','D'])
demand

A    30
B    50
C    10
D    20
dtype: int64

In [12]:
demand['A']

30

In [13]:
demand.loc['A']

30

In [14]:
demand.index

Index(['A', 'B', 'C', 'D'], dtype='object')

In [15]:
capacity = pd.Series([40]*3,index=[1,2,3])
capacity

1    40
2    40
3    40
dtype: int64

# Implementing the Formulation Incrementally

In [19]:
# Creating variables
from gurobipy import Model, GRB
mod = Model()
FCs = cost.index
regions = cost.columns
x = mod.addVars(FCs,regions,name='x')
mod.update()
x

{(1, 'A'): <gurobi.Var x[1,A]>,
 (1, 'B'): <gurobi.Var x[1,B]>,
 (1, 'C'): <gurobi.Var x[1,C]>,
 (1, 'D'): <gurobi.Var x[1,D]>,
 (2, 'A'): <gurobi.Var x[2,A]>,
 (2, 'B'): <gurobi.Var x[2,B]>,
 (2, 'C'): <gurobi.Var x[2,C]>,
 (2, 'D'): <gurobi.Var x[2,D]>,
 (3, 'A'): <gurobi.Var x[3,A]>,
 (3, 'B'): <gurobi.Var x[3,B]>,
 (3, 'C'): <gurobi.Var x[3,C]>,
 (3, 'D'): <gurobi.Var x[3,D]>}

In [20]:
cost

,A,B,C,D
1,20,18,21,8
2,8,23,24,8
3,25,8,8,19


In [23]:
sum(cost.loc[f,r]*x[f,r] for f in FCs for r in regions)

<gurobi.LinExpr: 20.0 x[1,A] + 18.0 x[1,B] + 21.0 x[1,C] + 8.0 x[1,D] + 8.0 x[2,A] + 23.0 x[2,B] + 24.0 x[2,C] + 8.0 x[2,D] + 25.0 x[3,A] + 8.0 x[3,B] + 8.0 x[3,C] + 19.0 x[3,D]>

In [24]:
capacity[1]

40

In [25]:
x[1,'A'] + x[1,'B'] + x[1,'C'] + x[1,'D'] <= 40

<gurobi.TempConstr: <gurobi.LinExpr: x[1,A] + x[1,B] + x[1,C] + x[1,D]> <= 40>

In [26]:
[x[1,r] for r in regions]

[<gurobi.Var x[1,A]>,
 <gurobi.Var x[1,B]>,
 <gurobi.Var x[1,C]>,
 <gurobi.Var x[1,D]>]

In [27]:
sum(x[1,r] for r in regions)<=capacity[1]

<gurobi.TempConstr: <gurobi.LinExpr: x[1,A] + x[1,B] + x[1,C] + x[1,D]> <= 40>

In [28]:
x[1,'A'] + x[2,'A'] + x[3,'A']>=30

<gurobi.TempConstr: <gurobi.LinExpr: x[1,A] + x[2,A] + x[3,A]> >= 30>

In [29]:
sum(x[f,'A'] for f in FCs)>=demand['A']

<gurobi.TempConstr: <gurobi.LinExpr: x[1,A] + x[2,A] + x[3,A]> >= 30>

In [30]:
# Full model
mod=Model()
x=mod.addVars(FCs,regions,name='x')
obj = sum(cost.loc[f,r]*x[f,r] for f in FCs for r in regions)
mod.setObjective(obj)
for f in FCs:
    mod.addConstr(sum(x[f,r] for r in regions)<=capacity[f],name=f'Capacity_{f}')
for r in regions:
    mod.addConstr(sum(x[f,r] for f in FCs)>=demand[r],name=f'Demand_{r}')
mod.write('10-supplyChain.lp')
!type 10-supplyChain.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  20 x[1,A] + 18 x[1,B] + 21 x[1,C] + 8 x[1,D] + 8 x[2,A] + 23 x[2,B]
   + 24 x[2,C] + 8 x[2,D] + 25 x[3,A] + 8 x[3,B] + 8 x[3,C] + 19 x[3,D]
Subject To
 Capacity_1: x[1,A] + x[1,B] + x[1,C] + x[1,D] <= 40
 Capacity_2: x[2,A] + x[2,B] + x[2,C] + x[2,D] <= 40
 Capacity_3: x[3,A] + x[3,B] + x[3,C] + x[3,D] <= 40
 Demand_A: x[1,A] + x[2,A] + x[3,A] >= 30
 Demand_B: x[1,B] + x[2,B] + x[3,B] >= 50
 Demand_C: x[1,C] + x[2,C] + x[3,C] >= 10
 Demand_D: x[1,D] + x[2,D] + x[3,D] >= 20
Bounds
End


In [31]:
# Numerical Solution
mod.setParam('OutputFlag',False)
mod.optimize()
print('Minimum cost:',mod.objVal)

Minimum cost: 1080.0


In [32]:
# Creating the output table
shipment=pd.DataFrame(index=FCs,columns=regions)
shipment

,A,B,C,D
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [33]:
shipment.loc[1,'A'] = 0
shipment

,A,B,C,D
1,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [34]:
# Filling in the output table
for f in FCs:
    for r in regions:
        shipment.loc[f,r]=x[f,r].x
shipment

,A,B,C,D
1,0.0,20.0,0.0,20.0
2,30.0,0.0,0.0,0.0
3,0.0,30.0,10.0,0.0
